# Teach an LLM to do additions

In [1]:
import torch
from torch import nn
from torch.nn import functional as F

import random
import math
import re
import time

In [2]:
number_bits = 3

dataset_size = 64_000
train_proportion = 0.9

log_interval = 200
batch_size = 64
epochs = 4
learning_rate = 1e-4

## Step 1: Construct a tokenizer

In [3]:
pad_token="[PAD]"
eos_token="[EOS]"

### Option 1: character-level

In [4]:
class character_level_tokenizer:
    """
    character-level
    """
    def __init__(self):
        self.vocab = [str(x) for x in range(10)] + ["+", "="] + [pad_token, eos_token]
        self.token_to_id = {v : k for k, v in enumerate(self.vocab)}
        self.id_to_token = {k : v for k, v in enumerate(self.vocab)}
        self.ntokens = len(self.vocab)
        self.pattern = f"[^{re.escape(''.join(self.vocab))}]"

    def clean(self, text):
        """
        removes all characters not in the vocabulary
        """
        out = re.sub(self.pattern, "", text)
        return out

    def pre_tokenization(self, text):
        """
        character-level
        """
        return [c for c in text]

    def encode(self, text):
        text_list = self.pre_tokenization(self.clean(text))
        return [self.token_to_id[c] for c in text_list]

    def decode(self, token_list):
        return "".join([self.id_to_token[x] for x in token_list])

In [5]:
tokenizer = character_level_tokenizer()
ntokens = tokenizer.ntokens
ntokens

14

In [6]:
prompt = "12 + 42 ="
inputs = tokenizer.encode(prompt)
inputs, tokenizer.decode(inputs)

([1, 2, 10, 4, 2, 11], '12+42=')

### Option 2: reversed character-level

In [7]:
class reversed_character_level_tokenizer:
    """
    reversed character-level
    """
    def __init__(self):
        self.vocab = [str(x) for x in range(10)] + ["+", "="] + [pad_token, eos_token]
        self.token_to_id = {v : k for k, v in enumerate(self.vocab)}
        self.id_to_token = {k : v for k, v in enumerate(self.vocab)}
        self.ntokens = len(self.vocab)
        self.pattern = f"[^{re.escape(''.join(self.vocab))}]"

    def clean(self, text):
        """
        removes all characters not in the vocabulary
        """
        out = re.sub(self.pattern, "", text)
        return out

    def pre_tokenization(self, text):
        output = []
        curr = ""
        i = 0
        while(i < len(text)):
            if text[i].isdigit():
                curr += text[i]
            else:
                if curr != "":
                    output.append(curr)
                curr = ""
                output.append(text[i])
            i += 1
        if curr != "":
            output.append(curr)
        return output

    def encode(self, text):
        text_list = self.pre_tokenization(self.clean(text))
        output = []
        for x in text_list:
            output += [self.token_to_id[c] for c in reversed(x)]
        return output

    def decode(self, token_list):
        output = ""
        i = 0
        while(i < len(token_list)):
            if token_list[i] >= 10:
                output += self.id_to_token[token_list[i]]
                i += 1
            else:
                number = []
                while(i < len(token_list) and token_list[i] < 10):
                    number.append(self.id_to_token[token_list[i]])
                    i += 1
                output += "".join(reversed(number))
        return output

In [8]:
tokenizer = reversed_character_level_tokenizer()
ntokens = tokenizer.ntokens
ntokens

14

In [9]:
prompt = "12 + 42 = "
inputs = tokenizer.encode(prompt)
inputs, tokenizer.decode(inputs)

([2, 1, 10, 2, 4, 11], '12+42=')

In [10]:
prompt_answer = "12 + 42 = 54"
inputs = tokenizer.encode(prompt_answer)
inputs, tokenizer.decode(inputs)

([2, 1, 10, 2, 4, 11, 4, 5], '12+42=54')

### Option 3: aligning numbers

In [11]:
class aligned_tokenizer:
    """
    aligns the numbers, from right to left
    """
    def __init__(self):
        self.vocab = [str(x) for x in range(10)] + ["+", "="] + ["F", "S"] + [pad_token, eos_token]
        self.token_to_id = {v : k for k, v in enumerate(self.vocab)}
        self.id_to_token = {k : v for k, v in enumerate(self.vocab)}
        self.ntokens = len(self.vocab)
        self.pattern = f"[^{re.escape(''.join(self.vocab))}]"

    def clean(self, text):
        """
        removes all characters not in the vocabulary
        """
        out = re.sub(self.pattern, "", text)
        return out

    def encode_prompt(self, num1, num2):
        max_length = max([len(num1), len(num2)])
        if len(num1) < max_length:
            num1 = num1 + "0" * (max_length - len(num1))
        if len(num2) < max_length:
            num2 = num2 + "0" * (max_length - len(num2))
        out = []
        for (x,y) in zip(num1, num2):
            out += [self.token_to_id["F"], self.token_to_id[x], self.token_to_id["S"], self.token_to_id[y]]
        return out

    def encode(self, text):
        text = self.clean(text)
        splits = re.split("[=+]", text)
        splits = [split for split in splits if len(split) > 0]
        if len(splits) == 1:
            # encoding an answer
            return [self.token_to_id[c] for c in text]
        if len(splits) == 2:
            # encoding a prompt
            return self.encode_prompt(splits[0][::-1], splits[1][::-1]) + [self.token_to_id["="]]
        if len(splits) == 3:
            # encoding a prompt + answer
            encoded_prompt = self.encode_prompt(splits[0][::-1], splits[1][::-1]) + [self.token_to_id["="]]
            encoded_answer = [self.token_to_id[c] for c in splits[2]] + [self.token_to_id[eos_token]]
            return encoded_prompt + encoded_answer

    def decode_prompt(self, token_list):
        i = 0
        num1 = ""
        num2 = ""
        while(i < len(token_list) and token_list[i] != self.token_to_id["="]):
            if token_list[i] == self.token_to_id[pad_token]:
                i += 1
                continue
            assert(self.id_to_token[token_list[i]] == "F")
            x = self.id_to_token[token_list[i+1]]
            assert(self.id_to_token[token_list[i+2]] == "S")
            y = self.id_to_token[token_list[i+3]]
            num1 += x
            num2 += y
            i += 4
        return (num1[::-1] + " + " + num2[::-1] + " = ", i + 1)

    def decode(self, token_list):
        if self.token_to_id["="] in token_list:
            output, i = self.decode_prompt(token_list)
        else:
            output, i = "", 0
        number = []
        while(i < len(token_list) and token_list[i] < 10):
            number.append(self.id_to_token[token_list[i]])
            i += 1
        output += "".join(number)
        return output

In [12]:
tokenizer = aligned_tokenizer()
ntokens = tokenizer.ntokens
ntokens

16

In [13]:
prompt = "12 + 42 = "
inputs = tokenizer.encode(prompt)
inputs, tokenizer.decode(inputs)

([12, 2, 13, 2, 12, 1, 13, 4, 11], '12 + 42 = ')

In [14]:
prompt_answer = "12 + 42 = 54"
inputs = tokenizer.encode(prompt_answer)
inputs, tokenizer.decode(inputs)

([12, 2, 13, 2, 12, 1, 13, 4, 11, 5, 4, 15], '12 + 42 = 54')

We choose the tokenizer to be used here:

In [15]:
# tokenizer = character_level_tokenizer()
# tokenizer = reversed_character_level_tokenizer()
tokenizer = aligned_tokenizer()

ntokens = tokenizer.ntokens
ntokens

16

## Step 2: Create a dataset for arithmetic operations

In [16]:
def sample_datapoint(number_bits = 3):
    """
    returns a string containing two random numbers on `number_bits` many bits and their sum.
    """
    a_list = [random.randint(0, 9) for _ in range(number_bits)]
    b_list = [random.randint(0, 9) for _ in range(number_bits)]
    a_int = int("".join([str(x) for x in a_list]))
    b_int = int("".join([str(x) for x in b_list]))
    sum_int = a_int + b_int
    return (str(a_int) + "+" + str(b_int) + "=", str(sum_int))

sample_datapoint(3)

('712+51=', '763')

In [17]:
data = []
for _ in range(dataset_size):
    data.append(sample_datapoint(number_bits))
data[:4]

[('5+32=', '37'),
 ('373+675=', '1048'),
 ('590+611=', '1201'),
 ('396+312=', '708')]

In [18]:
data_train = data[: int(train_proportion * dataset_size)]
data_test = data[int(train_proportion * dataset_size):]

len(data_train),len(data_test)

(57600, 6400)

## Step 3: Construct a model

In [19]:
class PositionalEncoding(nn.Module):
    r"""Inject some information about the relative or absolute position of the tokens in the sequence.
        The positional encodings have the same dimension as the embeddings, so that the two can be summed.
        Here, we use sine and cosine functions of different frequencies.
    .. math:
        \text{PosEncoder}(pos, 2i) = sin(pos/10000^(2i/d_model))
        \text{PosEncoder}(pos, 2i+1) = cos(pos/10000^(2i/d_model))
        \text{where pos is the word position and i is the embed idx)
    Args:
        d_model: the embed dim (required).
        dropout: the dropout value (default=0.1).
        max_len: the max. length of the incoming sequence (default=5000).
    """

    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        r"""Inputs of forward function
        Args:
            x: the sequence fed to the positional encoder model (required).
        Shape:
            x: [sequence length, batch size, embed dim]
            output: [sequence length, batch size, embed dim]
        """

        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)

In [20]:
class TransformerModel(nn.Transformer):
    def __init__(self, ntoken, ninp, nhead, nhid, nlayers, dropout=0.5):
        super(TransformerModel, self).__init__(d_model=ninp,
                                               nhead=nhead,
                                               dim_feedforward=nhid,
                                               num_encoder_layers=nlayers)
        self.input_emb = nn.Embedding(ntoken, ninp)
        self.pos_encoder = PositionalEncoding(ninp, dropout)
        self.decoder = nn.Linear(ninp, ntoken)

        self.ninp = ninp
        self.init_weights()

    def init_weights(self):
        initrange = 0.1
        nn.init.uniform_(self.input_emb.weight, -initrange, initrange)
        nn.init.zeros_(self.decoder.bias)
        nn.init.uniform_(self.decoder.weight, -initrange, initrange)

    def _generate_square_subsequent_mask(self, sz):
        return torch.log(torch.tril(torch.ones(sz,sz)))

    def forward(self, src):
        mask = self._generate_square_subsequent_mask(len(src)).to(device)
        self.src_mask = mask

        src = self.input_emb(src) * math.sqrt(self.ninp)
        src = self.pos_encoder(src)
        output_enc = self.encoder(src, mask=self.src_mask)
        output_dec = self.decoder(output_enc)
        return F.log_softmax(output_dec, dim=-1), output_enc

In [21]:
model = TransformerModel(ntoken = ntokens,
                         ninp = 128,
                         nhead = 16,
                         nhid = 64,
                         nlayers = 8)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
model.to(device)

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/transformer.py:379: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


cuda


TransformerModel(
  (encoder): TransformerEncoder(
    (layers): ModuleList(
      (0-7): 8 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=128, out_features=128, bias=True)
        )
        (linear1): Linear(in_features=128, out_features=64, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=64, out_features=128, bias=True)
        (norm1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (dropout2): Dropout(p=0.1, inplace=False)
      )
    )
    (norm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
  )
  (decoder): Linear(in_features=128, out_features=16, bias=True)
  (input_emb): Embedding(16, 128)
  (pos_encoder): PositionalEncoding(
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [22]:
def generate(model, prompts, new_tokens = 5):
    input_tensor = prompts # (length_prompts, batch_size)
    input_tensor = input_tensor.to(device)
    for _ in range(new_tokens):
        output, _ = model(input_tensor) # (length_prompts, batch_size, ntokens)
        last_output = output[-1,:,:] # (batch_size, ntokens)
        token = torch.argmax(last_output, -1).view((1,-1)) # (1, batch_size)
        input_tensor = torch.cat((input_tensor, token), 0)
    return input_tensor

In [35]:
model.eval()

prompt = "2+3="
prompt_tensor = torch.tensor(tokenizer.encode(prompt)).view((-1,1))
output = generate(model, prompt_tensor).view((1,-1))
output, tokenizer.decode(output.tolist()[0])

(tensor([[12,  2, 13,  3, 11, 13, 13, 13, 13, 13]], device='cuda:0'),
 '2 + 3 = ')

In [24]:
def pad(token_list, type_list = "prompts"):
    max_length = max([len(x) for x in token_list])
    out = []
    for x in token_list:
        if type_list == "prompts":
            out.append([tokenizer.token_to_id[pad_token]] * (max_length - len(x)) + x)
        if type_list == "answers":
            out.append(x + [tokenizer.token_to_id[eos_token]] + [tokenizer.token_to_id[pad_token]] * (max_length - len(x)))
    return out, max_length

In [25]:
prompts = [tokenizer.encode("1+1="), tokenizer.encode("21+35=")]
answers = [tokenizer.encode("2"), tokenizer.encode("56")]
padded_prompts, _ = pad(prompts, "prompts")
padded_answers, _ = pad(answers, "answers")
padded_prompts, padded_answers
[tokenizer.decode(p) for p in padded_prompts], [tokenizer.decode(p) for p in padded_answers]

(['1 + 1 = ', '21 + 35 = '], ['2', '56'])

In [26]:
def get_batch(split, i):
    data = data_train if split == 'train' else data_test
    prompts = [tokenizer.encode(data[i][0]) for i in range(i, i + batch_size)]
    padded_prompts, length_prompts = pad(prompts, "prompts")
    answers = [tokenizer.encode(data[i][1]) for i in range(i, i + batch_size)]
    padded_answers, length_answers = pad(answers, "answers")
    X = torch.stack([torch.tensor(x) for x in padded_prompts], 1)
    Y = torch.stack([torch.tensor(x) for x in padded_answers], 1)
    return X, Y, length_prompts, length_answers

In [27]:
X, Y, length_prompts, length_answers = get_batch("train", 243)
X.shape, Y.shape, length_prompts, length_answers

(torch.Size([13, 64]), torch.Size([5, 64]), 13, 4)

## Step 4: Evaluate

In [28]:
def evaluate():
    # Turn on evaluation mode disables dropout.
    model.eval()
    correct = 0.
    with torch.no_grad():
        for batch, i in enumerate(range(0, len(data_test) - 1, batch_size)):
            prompts, target_answers, length_prompts, length_answers = get_batch("test", i)
            prompts = prompts.to(device) # (length_prompts, batch_size)
            target_answers = target_answers.to(device) # (length_answers + 1, batch_size)
            output = generate(model, prompts, length_answers + 1) # (length_prompts + length_answers + 1, batch_size)
            answers_tokens = output[length_prompts:, :] # (length_answers + 1, batch_size), contains tokens
            equality_test = answers_tokens == target_answers # (length_answers + 1, batch_size), contains boolean values
            correct += torch.all(equality_test, axis=0).float().sum()
        accuracy = correct / len(data_test)
    return accuracy.item()

In [29]:
evaluate()

0.0

## Step 4: Train the model

In [30]:
def train_epoch():
    model.train()
    optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
    total_loss = 0.
    start_time = time.time()
    for batch, i in enumerate(range(0, len(data_train) - 1, batch_size)):
        prompts, target_answers, length_prompts, length_answers = get_batch("train", i)
        prompts = prompts.to(device) # (length_prompts, batch_size)
        target_answers = target_answers.to(device) # (length_answers, batch_size)
        input_tensor = torch.cat((prompts, target_answers), 0) # (length_prompts + length_answers, batch_size)
        model.zero_grad()
        output, _ = model(input_tensor) # (length_prompts + length_answers, batch_size, ntokens)
        output_answers = output[length_prompts-1:-1,:,:].reshape(-1, ntokens) # (length_answers * batch_size, ntokens)
        target_answers = target_answers.view(-1)
        loss = F.cross_entropy(output_answers, target_answers)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        if batch % log_interval == 0 and batch > 0:
            cur_loss = total_loss / log_interval
            elapsed = time.time() - start_time
            print('| {:5d}/{:5d} batches | ms/batch {:5.2f} | loss {:5.2f} | perplexity {:8.2f}'.format(batch, len(data_train) // batch_size,
                                                                                                        elapsed * 1000 / log_interval, cur_loss, math.exp(cur_loss)))
            total_loss = 0
            start_time = time.time()

def train():
    best_test_accuracy = None
    test_accuracy = evaluate()
    print('-' * 89)
    print('| initialisation | test accuracy {:5.2f}'.format(test_accuracy))
    print('-' * 89)
    for epoch in range(1, epochs+1):
        epoch_start_time = time.time()
        train_epoch()
        test_accuracy = evaluate()
        print('-' * 89)
        print('| end of epoch {:3d} | time: {:5.2f}s | test accuracy {:5.2f}'.format(epoch, (time.time() - epoch_start_time), test_accuracy))
        print('-' * 89)
        # Save the model if the test accuracy is the best we've seen so far.
        if not best_test_accuracy or test_accuracy < best_test_accuracy:
            with open("arithmetic.pt", 'wb') as f:
                torch.save(model, f)
            best_test_accuracy = test_accuracy

In [31]:
train()

-----------------------------------------------------------------------------------------
| initialisation | test accuracy  0.00
-----------------------------------------------------------------------------------------
|   200/  900 batches | ms/batch 21.59 | loss  2.17 | perplexity     8.78
|   400/  900 batches | ms/batch 25.82 | loss  1.74 | perplexity     5.71
|   600/  900 batches | ms/batch 21.20 | loss  1.62 | perplexity     5.07
|   800/  900 batches | ms/batch 21.80 | loss  1.55 | perplexity     4.69
-----------------------------------------------------------------------------------------
| end of epoch   1 | time: 23.68s | test accuracy  0.00
-----------------------------------------------------------------------------------------
|   200/  900 batches | ms/batch 21.30 | loss  1.48 | perplexity     4.40
|   400/  900 batches | ms/batch 24.46 | loss  1.44 | perplexity     4.21
|   600/  900 batches | ms/batch 22.57 | loss  1.41 | perplexity     4.08
|   800/  900 batches | ms/

In [32]:
model.eval()

for i in range(20):
    prompt, answers = data_test[i]
    prompt_tensor = torch.tensor(tokenizer.encode(prompt)).view((-1,1))
    output = generate(model, prompt_tensor, len(answers)).view((1,-1))
    print(tokenizer.decode(output.tolist()[0]) + "\t actual result: " + answers)

724 + 472 = 1111	 actual result: 1196
404 + 604 = 1000	 actual result: 1008
378 + 910 = 1218	 actual result: 1288
232 + 271 = 411	 actual result: 503
160 + 275 = 311	 actual result: 435
110 + 496 = 591	 actual result: 606
121 + 976 = 1011	 actual result: 1097
419 + 617 = 1000	 actual result: 1036
257 + 106 = 211	 actual result: 363
866 + 194 = 1000	 actual result: 1060
414 + 005 = 410	 actual result: 419
343 + 459 = 799	 actual result: 802
105 + 035 = 101	 actual result: 140
598 + 565 = 1110	 actual result: 1163
817 + 068 = 901	 actual result: 885
376 + 794 = 1100	 actual result: 1170
331 + 074 = 310	 actual result: 405
172 + 646 = 801	 actual result: 818
627 + 115 = 711	 actual result: 742
742 + 806 = 1611	 actual result: 1548


## Probing

In [33]:
import numpy as np

train_size = 1000
test_size = 100

model.eval()

def data_probing(size):
    X = []
    y = np.zeros(size)
    for i in range(size):
        input = torch.tensor(tokenizer.encode(data[i][0])).view((-1, 1)).to(device)
        _, output = model(input)
        output = output[-1,:,:].flatten()
        # determine whether there was a carry in the result:
        carry = len(data[i][1]) > len(data[i][0]) / 2
        X.append(output.cpu().detach().numpy())
        y[i] = carry
    return np.array(X), y

In [34]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

X_train, y_train = data_probing(train_size)
X_test, y_test = data_probing(test_size)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

reg = LogisticRegression()
reg.fit(X_train,y_train)
reg.score(X_test, y_test)

0.99